In [ ]:
import copy
import numpy as np
import pandas as pd

from astropy.time import Time

import matplotlib.pylab as plt

In [ ]:
# There ought to be a way to set the environment variable for this kernel
import os
os.environ["RUBIN_SIM_DATA_DIR"] = "/home/b/bechtol/rubin-user/rubin_sim_data"
os.getenv("RUBIN_SIM_DATA_DIR")

In [ ]:
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.scheduler.basis_functions import BaseBasisFunction, M5DiffBasisFunction, NotTwilightBasisFunction, NObsHighAmBasisFunction, FilterLoadedBasisFunction, OnceInNightBasisFunction, AvoidFastRevists
from rubin_scheduler.scheduler.detailers import DitherDetailer, CameraRotDetailer
from rubin_scheduler.scheduler.surveys import FieldSurvey
from rubin_scheduler.scheduler.schedulers import CoreScheduler
from rubin_scheduler.scheduler import  sim_runner
from rubin_scheduler.utils import ddf_locations

In [ ]:
# Note that can only set this date after downloading the full set sky brightness data
#mjd_start = Time('2024-04-01 00:00:00.000', format='iso').mjd
mjd_start = Time('2024-03-21 00:00:00.000', format='iso').mjd
mjd_start

In [ ]:
# We need the start date of the survey, so let's load up our model observatory and get that from the conditions
nside = 32  # Specify the HEALpix resolution
mo = ModelObservatory(nside=nside, ideal_conditions=True, mjd_start=mjd_start)
conditions = mo.return_conditions()

In [ ]:
#conditions.mjd

In [ ]:
#conditions.mounted_filters = ['g', 'r', 'i']

In [ ]:
#conditions.mounted_filters

In [ ]:
#dir(conditions)

In [ ]:
#conditions.current_filter

In [ ]:
#conditions.night

In [ ]:
#conditions.mjd

In [ ]:
#conditions.moon_ra, conditions.moon_dec

In [ ]:
#conditions.rot_tel_pos

In [ ]:
#np.nanmax(conditions.airmass)

In [ ]:
#print(conditions.site.latitude_rad, conditions.site.longitude_rad)
#print(conditions.site.latitude, conditions.site.longitude)
#assert np.radians(conditions.site.longitude) == conditions.site.longitude_rad
#assert np.radians(conditions.site.latitude) == conditions.site.latitude_rad

# Field Selection

In [ ]:
"""
# Start with a set of fields chosen to be uniformly spaced in RA so that some field should be visible throughout the night all year
#ra_array = np.arange(0., 360., 30.)
ra_array = np.arange(0., 360., 60.) + 5
dec_array = np.tile(-25., len(ra_array))
#dec_array = np.tile(-30., len(ra_array))
print(ra_array)
print(dec_array)
"""

In [ ]:
"""
targets = (
    (95., -25.), # High stellar densty, low extinction
    (125., -15.), # High stellar densty, low extinction
    #(360 - 145., -25.),
    (216, -12.5), # DEEP Solar Systen
    (360 - 110., 0.), # High stellar densty, low extinction
    #(219.80, -0.600), # DESI, GAMA, HSC DR2, KiDS-N
    #(360 - 53., -25),
    (270.891667, -30.033889), # Baade's Window
    (310, -19), # DEEP Solar System
    (9.45, -44.0),
    (35.708333, -4.75),
    (53.125, -28.1), # ECDFS
    (150.1, 2.1819444444444445), # COSMOS
    (58.9, -49.315), # EDFS_a
    (63.6, -47.6), # EDFS_b
)
"""
"""
# 2 January 2024
targets = (
    (95., -25.), # High stellar densty, low extinction
    (125., -15.), # High stellar densty, low extinction
    (179.60, 0.000), # DESI, GAMA, HSC DR2, KiDS-N
    #(360 - 145., -25.),
    (360 - 135., -40.), # High stellar densty, low extinction
    (216, -12.5), # DEEP Solar Systen
    (360 - 110., 2.), # High stellar densty, low extinction
    #(219.80, -0.600), # DESI, GAMA, HSC DR2, KiDS-N
    #(360 - 53., -25),
    #(270.891667, -30.033889), # Baade's Window
    (300., -41.), # High stellar densty, low extinction 
    (280., -48.), # High stellar densty, low extinction 
    (310, -19), # DEEP Solar System
    (9.45, -44.0), # ELAIS-S1 LSST DDF
    (35.708333, -4.75), # LSST DDF
    (53.125, -28.1), # ECDFS
    (150.1, 2.1819444444444445), # COSMOS
    (58.9, -49.315), # EDFS_a
    (63.6, -47.6), # EDFS_b
)


ra_array, dec_array = zip(*targets)
print(ra_array)
print(dec_array)
"""

In [ ]:
"""
ra_array = []
dec_array = []
for key in ['ELAISS1', 'XMM_LSS', 'ECDFS', 'COSMOS', 'EDFS_a']:
    ra_array.append(ddf_locations[key][0])
    dec_array.append(ddf_locations[key][1])

for key in ['A0', 'B0']:
    ra_array.append(deep_locations[key][0])
    dec_array.append(deep_locations[key][1])
"""

In [ ]:
"""
fields = {}
for ra, dec in zip(ra_array, dec_array):
    name = f"{int(ra)}{int(dec)}"
    fields[name] = {"RA": ra, "dec": dec}
print(fields)
"""

In [ ]:
import ops_rehearsal_fields
import importlib
importlib.reload(ops_rehearsal_fields)

name_fields, ra_fields, dec_fields = zip(*ops_rehearsal_fields.fields)

fields = {}
for name, ra, dec in zip(name_fields, ra_fields, dec_fields):
    fields[name] = {"RA": ra, "dec": dec}
for key in fields:
    print(key, fields[key]["RA"], fields[key]["dec"])

In [ ]:
import healpy as hp

In [ ]:
m_stellar_density = np.load('/home/b/bechtol/rubin_sim_data/maps/StarMaps/starDensity_g_nside_64.npz')

In [ ]:
hp.mollview(np.log10(m_stellar_density['starDensity'][:,20]), title='log10(stellar density)')
hp.projscatter(ra_fields, dec_fields, lonlat=True, color='red')

In [ ]:
#m_extinction = np.load('/home/b/bechtol/rubin_sim_data/maps/DustMaps/dust_nside_32.npz')

In [ ]:
#hp.mollview(m_extinction['ebvMap'], min=0, max=0.2, title='E(B-V)')
#hp.projscatter(ra_fields, dec_fields, lonlat=True, color='red')

# Basis Functions

In [ ]:
from rubin_scheduler.scheduler import features
from rubin_scheduler.scheduler.utils import IntRounded

class AvoidFastRevistsBasisFunction(BaseBasisFunction):
    """Marks targets as unseen if they are in a specified time window
    in order to avoid fast revisits.

    Parameters
    ----------
    filtername: `str` ('r')
        The name of the filter for this target map.
    gap_min : `float` (25.)
        Minimum time for the gap (minutes).
    nside: `int` (default_nside)
        The healpix resolution.
    penalty_val : `float` (np.nan)
        The reward value to use for regions to penalize.
        Will be masked if set to np.nan (default).
    """

    def __init__(self, survey_name=None, gap_min=30.0, penalty_val=np.nan):
        super(AvoidFastRevistsBasisFunction, self).__init__()

        self.penalty_val = penalty_val

        self.gap_min = IntRounded(gap_min / 60.0 / 24.0)

        self.survey_features = dict()
        self.survey_features["LastObservation"] = features.LastObservation(survey_name)

    def _calc_value(self, conditions, indx=None):
        diff = IntRounded(conditions.mjd - self.survey_features["LastObservation"].feature['mjd'])
        if diff < self.gap_min:
            result = self.penalty_val
        else:
            result = 0.
        return result

In [ ]:
#bfs = AvoidFastRevistsBasisFunction(survey_name='test')
#conditions.mjd - bfs.survey_features["LastObservation"].feature['mjd'][0]

In [ ]:
class RisingBasisFunction(BaseBasisFunction):
    """Reward fields that are rising.

    Parameters
    ----------
    RA : float
        The RA of the point in the sky (degrees)
    """

    def __init__(self, RA, weight=0.1):
        super(RisingBasisFunction, self).__init__()
        self.ra_hours = RA * 24 / 360.0
        self.weight = weight

    # Probably not needed
    def check_feasibility(self, conditions):
        result = True
        return result

    def _calc_value(self, conditions, indx=None):
        hour_angle = conditions.lmst - self.ra_hours
        return -1 * self.weight * hour_angle

In [ ]:
#from astropy.coordinates import SkyCoord, angular_separation

In [ ]:
#ra_1, dec_1 = 180., 0.
#ra_2, dec_2 = 181., 1.

#angsep = angular_separation(np.radians(ra_1), np.radians(dec_1), np.radians(ra_2), np.radians(dec_2))
#print(np.degrees(angsep))

In [ ]:
from astropy.coordinates import angular_separation

class AvoidMoonBasisFunction(BaseBasisFunction):
    """Only execute if angular separation to the Moon is above a specified threshold.

    Parameters
    ----------
    ra : float
        The RA of the point in the sky (degrees)
    dec : float
        The declination of the point in the sky (degrees)
    min_moon_dist : float
        The minimum angular separation to the moon (degrees)
    """

    def __init__(self, ra, dec, min_moon_dist=20.):
        super(AvoidMoonBasisFunction, self).__init__()
        self.ra = ra
        self.dec = dec
        self.min_moon_dist = min_moon_dist

    def check_feasibility(self, conditions):
        moon_dist = np.degrees(
            angular_separation(
                np.radians(self.ra), np.radians(self.dec), conditions.moon_ra, conditions.moon_dec
            )
        )
        if moon_dist > self.min_moon_dist:
            result = True
        else:
            result = False
        return result

In [ ]:
from rubin_scheduler.utils import approx_ra_dec2_alt_az

class MaximumAirmassBasisFunction(BaseBasisFunction):
    """Only execute if the airmass is a below a specified threshold.

    Parameters
    ----------
    ra : float
        The RA of the point in the sky (degrees)
    dec : float
        The declination of the point in the sky (degrees)
    max_airmass : float
        The maximum airmass (degrees)
    """

    def __init__(self, ra, dec, max_airmass=2.0):
        super(MaximumAirmassBasisFunction, self).__init__()
        self.ra = ra
        self.dec = dec
        self.max_airmass = max_airmass

    def check_feasibility(self, conditions):
        alt, az = approx_ra_dec2_alt_az(
            self.ra,
            self.dec,
            conditions.site.latitude,
            conditions.site.longitude,
            conditions.mjd,
        )
        
        airmass = 1. / np.cos(np.radians(90. - alt))
        
        if airmass < self.max_airmass:
            result = True
        else:
            result = False

        #import pdb; pdb.set_trace()
        
        return result

In [ ]:
from rubin_scheduler.scheduler.detailers import BaseDetailer
from rubin_scheduler.utils import _approx_ra_dec2_alt_az, _approx_altaz2pa

class CameraRotPerObservationListDetailer(BaseDetailer):
    """
    Randomly set the camera rotation for each observation list.

    Parameters
    ----------
    max_rot : `float` (90.)
        The maximum amount to offset the camera (degrees)
    min_rot : `float` (90)
        The minimum to offset the camera (degrees)
    """

    def __init__(self, max_rot=90.0, min_rot=-90.0, seed=42): # nnights=7305):
        self.survey_features = {}

        self.current_night = -1
        self.max_rot = np.radians(max_rot)
        self.min_rot = np.radians(min_rot)
        self.range = self.max_rot - self.min_rot
        self.seed = seed
        #self.rng = np.random.default_rng(seed)
        #self.offsets = self.rng.random(100 * nnights)
        self.offset = None

    def _generate_offsets_orig(self, n_offsets, night, mjd):
        #print(mjd)
        #print(np.asarray(mjd).item() % 100)
        mjd_hash = round(100 * (np.asarray(mjd).item() % 100))
        #print(mjd_hash)
        rng = np.random.default_rng(mjd_hash * self.seed)
        #print(night)
        #print(type(night))
        #print(mjd)
        #print(type(mjd))
        #index = 1
        #index = round(
        #    (100 * night) + (100. * (mjd % 100))
        #)
        #self.offset = self.offsets[index] * self.range + self.min_rot
        self.offset = (rng.random() * self.range) + self.min_rot
        offsets = np.ones(n_offsets) * self.offset
        return offsets

    def _generate_offsets_filter_change(self, filter_list, mjd, initial_offset):
        """Generate a random camera rotation for each filter change.
        """
        
        mjd_hash = round(100 * (np.asarray(mjd).item() % 100))
        rng = np.random.default_rng(mjd_hash * self.seed)
        
        offsets = np.zeros(len(filter_list))
        offset = np.asarray(initial_offset).item()
        offsets[0] = offset
        
        for ii in range(1, len(offsets)):            
            if filter_list[ii] != filter_list[ii - 1]:
                # Filter change
                offset = (rng.random() * self.range) + self.min_rot
            offsets[ii] = offset
        
        return offsets
        
    def __call__(self, observation_list, conditions):
        # Generate offsets in camera rotator
        #offsets = self._generate_offsets_orig(len(observation_list), conditions.night, conditions.mjd)
        #import pdb; pdb.set_trace()
        #offsets = self._generate_offsets(observation_list, conditions.night, conditions.mjd)

        filter_list = [np.asarray(obs['filter']).item() for obs in observation_list]
        offsets = self._generate_offsets_filter_change(filter_list, conditions.mjd, conditions.rot_tel_pos)
        
        for i, obs in enumerate(observation_list):
            alt, az = _approx_ra_dec2_alt_az(
                obs["RA"],
                obs["dec"],
                conditions.site.latitude_rad,
                conditions.site.longitude_rad,
                conditions.mjd,
            )
            obs_pa = _approx_altaz2pa(alt, az, conditions.site.latitude_rad)
            obs["rotSkyPos"] = (offsets[i] - obs_pa) % (2.0 * np.pi)
            obs["rotTelPos"] = offsets[i]

        return observation_list

# Scheduler

In [ ]:
# Basis functions
#bfs = []

sun_alt_limit = -12.0 # deg

bfs = [
    M5DiffBasisFunction(filtername='r', nside=nside),
    NotTwilightBasisFunction(sun_alt_limit=sun_alt_limit),
    #AvoidFastRevistsBasisFunction(filtername='r', nside=nside, gap_min=25.0, penalty_val=np.nan),
    #FilterLoadedBasisFunction(filternames=['g', 'r', 'i']),
    #OnceInNightBasisFunction(notes=['dense_dithered']),
    #NObsHighAmBasisFunction(),
]

In [ ]:
#bfs[2].check_feasibility(conditions)

In [ ]:
#rng = np.random.default_rng(42)
#for ii in range(0, 10):
#    print(rng.random())

In [ ]:
# Add in detailer
#detailers = None
detailers = [
    DitherDetailer(max_dither=0.2, per_night=False),
    CameraRotPerObservationListDetailer(),
    #CameraRotDetailer(per_night=True),
]

In [ ]:
nvis_master = [20, 20, 20]
sequence = "gri"
exptime = 30
u_exptime = 30
reward_value = None
nexp = 2

min_moon_dist = 20. # deg
max_airmass = 2.0

surveys = []

for survey_name in fields.keys():

    bfs_survey = copy.deepcopy(bfs)
    bfs_survey.append(RisingBasisFunction(RA=fields[survey_name]["RA"]))
    bfs_survey.append(AvoidFastRevistsBasisFunction(survey_name=survey_name))
    bfs_survey.append(AvoidMoonBasisFunction(
        ra=fields[survey_name]["RA"], 
        dec=fields[survey_name]["dec"], 
        min_moon_dist=min_moon_dist)
    )
    bfs_survey.append(MaximumAirmassBasisFunction(
        ra=fields[survey_name]["RA"], 
        dec=fields[survey_name]["dec"],
        max_airmass=max_airmass)
    )
    
    print(survey_name)
    #reward_value = 100 if fields[survey_name]["RA"] == 150. else 0
    #reward_value = 100.
    #reward_value = np.random.random()
    #reward_value = None
    surveys.append(
        #DeepDrillingSurvey(
        FieldSurvey(
            bfs_survey,
            fields[survey_name]["RA"],
            fields[survey_name]["dec"],
            sequence=sequence,
            nvis=nvis_master,
            exptime=exptime,
            u_exptime=u_exptime,
            survey_name=survey_name,
            reward_value=reward_value,
            nside=nside,
            nexp=nexp,
            detailers=detailers,
        )
    )

In [ ]:
#surveys[0].basis_functions

In [ ]:
scheduler = CoreScheduler(surveys, nside=nside)

In [ ]:
# Note the deep copy used here. The function sim_runner modifies the observatory
new_mo, new_scheduler, observations = sim_runner(copy.deepcopy(mo), scheduler, survey_length=21., verbose=True)
del new_mo
del new_scheduler

In [ ]:
df = pd.DataFrame(np.hstack(observations))
df

In [ ]:
df.columns

In [ ]:
assert np.all(observations['airmass'] < max_airmass)
assert np.all(np.degrees(observations['moonDist']) > min_moon_dist)

In [ ]:
print(np.sum(observations['airmass'] > 2.0))
print(np.max(observations['airmass']))

In [ ]:
print(np.min(np.degrees(observations['moonDist'])))

In [ ]:
np.unique(observations['note'])

In [ ]:
np.unique(observations['filter'])

# Quick Evaluation

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['alt'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('MJD')
plt.ylabel('Altitude (degrees)')

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], observations['airmass'][in_filt], 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('MJD')
plt.ylabel('Airmass')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], observations['slewtime'][in_filt], 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.xlim(observations['mjd'][0], observations['mjd'][500])

In [ ]:
plt.figure(dpi=200)
plt.scatter(observations['mjd'], np.degrees(observations['rotSkyPos']), c=np.degrees(observations['RA']), s=1)
plt.colorbar(label='RA (deg)')

plt.xlabel('MJD')
plt.ylabel('rotSkyPos')

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
#plt.scatter(observations['mjd'], np.degrees(observations['rotTelPos']), c=np.degrees(observations['RA']), s=1)
#plt.colorbar(label='RA (deg)')

for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['rotTelPos'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()

plt.xlabel('MJD')
plt.ylabel('rotTelPos')
plt.xlim(observations['mjd'][0], observations['mjd'][500])

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(np.degrees(observations['RA'][in_filt]), observations['airmass'][in_filt], 
                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.legend()
plt.xlabel('RA (deg)')
plt.ylabel('Airmass')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['RA'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.legend()
plt.xlabel('MJD')
plt.ylabel('RA (deg)')

In [ ]:
plt.figure(dpi=200)
#for filtername in f2c:
#    in_filt = np.where(observations['filter'] == filtername)[0]
#    if in_filt.size > 0:
#        plt.plot(observations['mjd'][in_filt], np.degrees(observations['moonDist'][in_filt]), 
#                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.scatter(observations['mjd'], np.degrees(observations['moonDist']), c=observations['moonPhase'], s=1)
plt.colorbar(label='moonPhase')

#plt.legend()
plt.xlabel('MJD')
plt.ylabel('moonDist (deg)')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(np.degrees(observations['RA'][in_filt]), np.degrees(observations['rotSkyPos'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.legend()
plt.xlabel('RA (deg)')
plt.ylabel('rotSkyPos')

In [ ]:
gap = (np.diff(observations['mjd']) * 24. * 3600.)

plt.figure()
plt.yscale('log')
bins = np.linspace(0., 300., 101)
plt.hist(gap, bins=bins)
plt.ylabel('Counts')
plt.xlabel('Time Gap (s)')

selection = gap > 300.
for index in np.nonzero(selection)[0]:
    print(gap[index], observations['mjd'][index + 1])

# Write the output

In [ ]:
import sqlite3

In [ ]:
# open up a connection to a new database
conn = sqlite3.connect('ops_rehearsal_apr_2024.db')

In [ ]:
df.to_sql('observations', conn, index=False, if_exists='replace')